### Get tweets with GU-related hashtags into a dataframe

In [78]:
import tweepy
import json
import pandas as pd
import csv
import re 
from textblob import TextBlob
import string
import preprocessor as p
from datetime import datetime

# Twitter credentials for the app
consumer_key = 'PUJGbYNh7t5Wdf19pid0F339i'
consumer_secret = 'TkiY2twuL5DsNzHkcYO1Cyv1xXGXi05qGTMbhDIcIsQUKnL1PD'
access_key= '1026361616604913664-8gQFpa8Ea6hpUPTgr1vdGHxwb4yqKE'
access_secret = 'bhN75eHt2GCFGN5slOpZCam4nMNKQVgqK82AbQaFvLwVX'

# Pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

# Set dataframe columns
COLS = ['hashtag','created_at','text','source','geo','user_id','user_location','user_followers','user_friends','retweet']


In [80]:
# Fetch tweets, save to csv read as dataframe
def ForEachHashtag(hashtag):
    # Open/Create a file to append data
    csvFile = open("{}{}".format(hashtag, ".csv"), 'w')
    
    #Use csv Writer
    csvWriter = csv.writer(csvFile)
    i=0
    for tweet in tweepy.Cursor(api.search,q="#"+hashtag, count = 100, lang="en",).items():
        #print(i,end='\r')
        csvWriter.writerow([hashtag,tweet.created_at, tweet.text.encode('utf-8'), tweet.source, 
                             tweet.geo, tweet.user.id, tweet.user.location,tweet.user.followers_count, 
                             tweet.user.friends_count, tweet.retweet_count])
        i=i+1
    df = pd.read_csv("{}{}".format(hashtag, ".csv"), header = None)
    df.columns = COLS
    df = df[df['text'].notna()]
    return(df)
    
#ForEachHashtag("hoya")

In [83]:
def FetchAllHashtags(list_of_hashtags):
    df = pd.DataFrame(columns = COLS)
    list_of_df = []
    for i in list_of_hashtags:
        print(i)
        list_of_df.append(ForEachHashtag(i))
    res = pd.concat(list_of_df)
    res.to_csv(r"big_df.csv", index = None, header = True)
    return(res)

big_df = FetchAllHashtags(['hoya','hoyas','georgetown','georgetownuniversity','hoyasaxa'])  #'jackthebulldog'


hoya
hoyas
georgetown
georgetownuniversity
hoyasaxa


In [84]:
big_df

,hashtag,created_at,text,source,geo,user_id,user_location,user_followers,user_friends,retweet
0,hoya,2020-02-21 13:32:32,b'RT @HowonUpdates: [VID] 200220 Kiseop Instag...,Twitter for Android,NaN,3478632974,NaN,88,358,60
1,hoya,2020-02-21 13:31:53,b'RT @HowonUpdates: [VID] 200220 Kiseop Instag...,Twitter Web App,NaN,861569795904258049,NaN,429,2128,60
2,hoya,2020-02-21 10:12:07,b'RT @ChewieKpop: INFINITE DESTINY in America\...,Twitter for iPhone,NaN,1126491750263132160,Japan,164,80,5
3,hoya,2020-02-21 09:51:15,b'RT @HowonUpdates: [VID] 200220 Kiseop Instag...,Twitter for Android,NaN,779386332628021249,NaN,539,527,60
4,hoya,2020-02-21 07:48:33,"b""RT @HowonUpdates: [INFO] 200220 #HOYA replie...",Twitter for Android,NaN,1094398367868674048,Cebu_인피니트,282,562,36
...,...,...,...,...,...,...,...,...,...,...
1109,hoyasaxa,2020-02-12 16:35:26,b'Meeting &amp; befriending @Briebrunnin remai...,Twitter Web App,NaN,3298774073,NaN,10893,3130,0
1110,hoyasaxa,2020-02-12 16:33:57,b'#startupgrind2020 with the DMV and @Halcyon...,Instagram,"{'type': 'Point', 'coordinates': [37.4849, -12...",87440584,"Arlington, VA",84,167,0
1111,hoyasaxa,2020-02-12 15:51:39,b'RT @HoyasMLacrosse: Check out the new white ...,Twitter Web App,NaN,3143645973,"Ellicott City, MD",91,380,17
1112,hoyasaxa,2020-02-12 15:15:44,b'RT @JulianDionG: The steps at Savoy Elementa...,Twitter for iPhone,NaN,883846221185642496,"North Miami, FL",49,136,1


### Text analysis

In [ ]:
# sentiment analysis with NLTK and Spacy (linguistic)





#list of words I have seen
seenit=[]
#dict of word counts
WordDict={}
Rawfilename="TwitterResultsRaw.txt"
Freqfilename="TwitterWordFrq.txt"


#FILE=open(Freqfilename,"w")
#FILE2=open(Rawfilename, "w")
R_FILE=open(Rawfilename,"w")
F_FILE=open(Freqfilename, "w")

IgnoreThese=["and", "And", "AND","THIS", "This", "this", "for", "FOR", "For", 
             "THE", "The", "the", "is", "IS", "Is", "or", "OR", "Or", "will", 
             "Will", "WILL", "God", "god", "GOD", "Bible", "bible", "BIBLE",
             "CanChew", "Download", "free", "FREE", "Free", "will", "WILL", 
             "Will", "hits", "hit", "within", "steam", "Via", "via", "know", "Study",
             "study", "unit", "Unit", "always", "take", "Take", "left", "Left",
             "lot","robot", "Robot", "Lot", "last", "Last", "Wonder", "still", "Still",
             "ferocious", "Need", "need", "food", "Food", "Flint", "MachineCredit",
             "Webchat", "luxury", "full", "fifdh17", "New", "new", "Caroline",
             "Tirana", "Shuayb", "repro", "attempted", "key", "Harrient", 
             "Chavez", "Women", "women", "Mumsnet", "Ali", "Tubman", "girl","Girl",
             "CSW61", "IWD2017", "Harriet", "Great", "great", "single", "Single", 
             "tailoring", "ask", "Ask"]
###Look at the words
for w in BigBag:
    if(w not in IgnoreThese):
        rawWord=w+" "
        R_FILE.write(rawWord)
        if(w in seenit):
            #print(w, seenit)
            WordDict[w]=WordDict[w]+1 #increment the times word is seen
        else:
            ##add word to dict and seenit
            seenit.append(w)
            WordDict[w]=1
    
#print(WordDict)  
#print(seenit)
#print(BagOfWords)



for key in WordDict:
    #print(WordDict[key])
    if(WordDict[key]>1):
        if(key not in IgnoreThese):
            #print(key)
            Key_Value=key + "," + str(WordDict[key]) + "\n"
            F_FILE.write(Key_Value)


#FILE.close()
#FILE2.close()
R_FILE.close()
F_FILE.close()


### Other analyses

In [ ]:
# classification/regression, cluster, network, statistical models, neural net with word2vec etc.